In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

## Install libraries

In [2]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.0.17 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 2.12.5 which is incompatible.
spacy 3.2.0 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 2.12.5 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
You should consider upgrading via the 'C:\Users\ankus\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings

# Langchain has youtubeloader as well but its bit buggy for now

c:\Users\ankus\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 1a - Indexing (Document Ingestion)

In [8]:
video_id = "Gfr50f6ZBvo" # only the ID, not full URL
languages=['de', 'en']
try:
    # If you don’t care which language, this returns the “best” one
    transcript_list = YouTubeTranscriptApi().fetch(video_id)
    transcript_list = list(transcript_list)  # ensure it's a sequence
    # Flatten robustly (handles dicts or objects with .text)
    texts = []
    for item in transcript_list:
        if isinstance(item, dict):
            texts.append(item.get("text", ""))
        else:
            texts.append(getattr(item, "text", ""))
    transcript = " ".join(t for t in texts if t)
    print(transcript[:1000])

except TranscriptsDisabled:
    print("No captions available for this video.")

the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get good enough 

In [45]:
transcript_list

[FetchedTranscriptSnippet(text='the following is a conversation with', start=0.08, duration=3.44),
 FetchedTranscriptSnippet(text='demus hasabis', start=1.76, duration=4.96),
 FetchedTranscriptSnippet(text='ceo and co-founder of deepmind', start=3.52, duration=5.119),
 FetchedTranscriptSnippet(text='a company that has published and builds', start=6.72, duration=4.48),
 FetchedTranscriptSnippet(text='some of the most incredible artificial', start=8.639, duration=4.561),
 FetchedTranscriptSnippet(text='intelligence systems in the history of', start=11.2, duration=4.8),
 FetchedTranscriptSnippet(text='computing including alfred zero that', start=13.2, duration=3.68),
 FetchedTranscriptSnippet(text='learned', start=16.0, duration=2.96),
 FetchedTranscriptSnippet(text='all by itself to play the game of gold', start=16.88, duration=4.559),
 FetchedTranscriptSnippet(text='better than any human in the world and', start=18.96, duration=5.6),
 FetchedTranscriptSnippet(text='alpha fold two that s

In [46]:
transcript


"the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get good enough

## Step 1b - Indexing (Text Splitting)

In [9]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [48]:
len(chunks)

168

In [49]:
chunks[100]

Document(metadata={}, page_content="and and kind of come up with descriptions of the electron clouds where they're gonna go how they're gonna interact when you put two elements together uh and what we try to do is learn a simulation uh uh learner functional that will describe more chemistry types of chemistry so um until now you know you can run expensive simulations but then you can only simulate very small uh molecules very simple molecules we would like to simulate large materials um and so uh today there's no way of doing that and we're building up towards uh building functionals that approximate schrodinger's equation and then allow you to describe uh what the electrons are doing and all materials sort of science and material properties are governed by the electrons and and how they interact so have a good summarization of the simulation through the functional um but one that is still close to what the actual simulation would come out with so what um how difficult is that to ask w

## Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [ ]:
# pip install torch --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ankus\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import numpy as np
import torch

print(np.__version__)
print(torch.__version__)
print(torch.tensor([1,2,3]).numpy())

1.26.4
2.8.0+cpu
[1 2 3]


In [10]:
import numpy as Numpy 
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, embeddings)

C:\Users\ankus\AppData\Local\Temp\ipykernel_4412\3764330337.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\ankus\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [11]:
vector_store.index_to_docstore_id

{0: '6d40b8a1-02ae-448b-baa2-b9f6fa3b4b38',
 1: 'a594c514-f5ed-4186-9689-fa44dc9674b6',
 2: '31969ef9-6eea-4f31-8072-7bc32ddb2fe7',
 3: '5e76edaf-5dc7-4176-a07b-ab782cfeab9b',
 4: '42f8367c-b97a-4cce-a7cd-64d7df804e33',
 5: 'd7ce7bf6-eeb4-4228-b548-0bab0b096791',
 6: 'a75df9fb-350a-45fd-8371-bea5e79bedcd',
 7: '586cd238-9c3b-46be-81c6-086b68e1db55',
 8: '54d53a51-dbf9-4b75-a608-5b46260465e4',
 9: '39729aae-6b66-40d9-bab4-7496d2eb69be',
 10: 'c95264d5-25cb-4a3f-865d-6c5df84c0a70',
 11: 'ec6c26ac-5556-4e16-afd1-94791ce47db2',
 12: '9f568622-3111-40da-a125-ab0248bf2e52',
 13: 'cd85b365-e7b7-46e0-ad3a-0c6a8495d40f',
 14: '6ad03030-ee5f-4a74-afae-2e6fa23a0808',
 15: '1ccfd71d-1fe6-48df-a000-f570b72a2707',
 16: '44374798-0c42-4d8a-9700-fbe6c5bb22df',
 17: '43b147e3-8005-4628-82c7-82d3801d27c4',
 18: 'd7bc9a72-ea83-4016-8eaa-6db3e46e593c',
 19: 'c50591dc-a7fc-4b02-9780-69aab22ae17a',
 20: '7f8bc56c-3042-499a-a2ef-546c7e30da2e',
 21: '68551767-619d-431f-8f0b-75d1accdf861',
 22: '4e02e91d-2299-

In [13]:
vector_store.get_by_ids(['affc1e54-b68b-4bed-b8d5-814769372e79'])

[Document(id='affc1e54-b68b-4bed-b8d5-814769372e79', metadata={}, page_content="are repeatable um it feels like it's almost structured in a way to be conducive to gaining knowledge so i feel like and you know why should computers be even possible isn't that amazing that uh computational electronic devices can can can can be possible and they're made of sand our most you know common element that we have you know silicon that on the on the earth's crust they could be made of diamond or something then we would have only had one computer yeah right so it's a lot of things are kind of slightly suspicious to me it sure as heck sounds this puzzle sure sounds like something we talked about earlier what it takes to to design a game that's really fun to play for prolonged periods of time and it does seem like this puzzle like you mentioned the more you learn about it the more you realize how little you know so it humbles you but excites you by the possibility of learning more it's one heck of a 

## Step 2 - Retrieval

In [14]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [15]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A905EE2B50>, search_kwargs={'k': 4})

In [16]:
retriever.invoke('What is deepmind')

[Document(id='e5f5cd24-5235-4280-bb28-969657372483', metadata={}, page_content="and how it works this is tough to uh ask you this question because you probably will say it's everything but let's let's try let's try to think to this because you're in a very interesting position where deepmind is the place of some of the most uh brilliant ideas in the history of ai but it's also a place of brilliant engineering so how much of solving intelligence this big goal for deepmind how much of it is science how much is engineering so how much is the algorithms how much is the data how much is the hardware compute infrastructure how much is it the software computer infrastructure yeah um what else is there how much is the human infrastructure and like just the humans interact in certain kinds of ways in all the space of all those ideas how much does maybe like philosophy how much what's the key if um uh if if you were to sort of look back like if we go forward 200 years look back what was the key 

## Step 3 - Augmentation

In [61]:
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(
    model="llama3",
    temperature=0.2
)

In [62]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [63]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [64]:
retrieved_docs

[Document(id='d2c4ccde-1fb1-4c4d-bb6c-35e6ee44334c', metadata={}, page_content="in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be intere

In [65]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be interesting from a research perspective from our point of view from an ai point of\n\

In [66]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [67]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we 

## Step 4 - Generation

In [68]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of nuclear fusion is discussed in this video. Specifically:

* The speaker mentions collaborating with EPFL (École polytechnique fédérale de Lausanne) in Switzerland to use their test reactor for fusion research.
* They discuss solving one of the challenges in fusion, which was holding a plasma in specific shapes and containing it for a record amount of time. This was achieved using a controller that can contain and hold the plasma in different shapes.
* The speaker mentions looking at other problems in the fusion area to tackle next.
* They also mention a paper on magnetic control of tokamak plasmas using deep reinforcement learning, which is an attempt to solve nuclear fusion using AI.

So, the topic of nuclear fusion is discussed as a potential application of AI and machine learning.


## Building a Chain

In [69]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [70]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [71]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [72]:
parallel_chain.invoke('who is Demis')

{'context': "to get world peace because there's also other corrupting things like wanting power over people and this kind of stuff which is not necessarily satisfied by by just abundance but i think it will help um and i think uh but i think ultimately ai is not going to be run by any one person or one organization i think it should belong to the world belong to humanity um and i think maybe many there'll be many ways this will happen and ultimately um everybody should have a say in that do you have advice for uh young people in high school and college maybe um if they're interested in ai or interested in having a big impact on the world what they should do to have a career they can be proud of her to have a life they can be proud of i love giving talks to the next generation what i say to them is actually two things i i think the most important things to learn about and to find out about when you're when you're young is what are your true passions is first of all there's two things on

In [73]:
parser = StrOutputParser()

In [74]:
main_chain = parallel_chain | prompt | llm | parser

In [75]:
main_chain.invoke('Can you summarize the video')

"The transcript appears to be from an interview or conversation between Demas and someone else. The topic of discussion is how to explain complex concepts, particularly in physics, in a simple and clear manner.\n\nDemas suggests that a deeper understanding of physics might require a fundamental re-examination of the subject, encompassing mysteries such as consciousness, life, and gravity. He also mentions the importance of being able to explain complex topics simply, citing Richard Feynman as an example.\n\nThe conversation touches on the topic of artificial intelligence (AI) and its potential to enhance human capabilities. Demas shares a personal anecdote about Claude Shannon's original chess program from 1949 and the famous match between Deep Blue and Kasparov in 1997.\n\nOverall, the conversation is about the importance of clear communication, the limitations of current understanding in physics, and the potential for AI to augment human intelligence."